### 合并数据集
> **pd.merge(left,right,on='key',suffixes=('_left','_right'))的各个参数含义**
>- left : 参与合并的左侧DataFrame
>- right : 参与合并的右侧的DataFrame
>- how ：inner、outer、left、right其中之一，默认是inner
>- on : 用于连接的列名。必须存在于左右两个DataFrame对象中
>- left_on ：左侧DataFrame 用作连接键的列
>- right_on ：右侧DataFrame中用作连接的键
>- left_index ：将左侧的索引用作其连接键
>- right_index ： 将右侧的的索引用作其连接键
>- sort ：根据连接键对合并后的数据进行排序，默认为True，（有时处理大数据集是一般禁用）
>- suffixes ：字符串值元祖，用于追加重叠列名的末尾
>- copy : 默认设置为False

#### 合并的df存在重名的情况

In [4]:
import pandas as pd
l_df= pd.DataFrame({
    'key1':['Shanghai','Beijing','Guangzhou'],
     'Key2':["Hanzghou","Wuhan","Chengdu"],
    'Value':[45,54,67] 
        })
r_df = pd.DataFrame({
    'key1':['Shanghai','Beijing','Chenzhen','Guanzghou'],
    'Key2':["Hanzghou","Wuhan","Chengdu","Zhengzhou"],
    'Value':[45,45,78,90] 
})

In [6]:
pd.merge(l_df,r_df,on='key1')

,key1,Key2_x,Value_x,Key2_y,Value_y
0,Shanghai,Hanzghou,45,Hanzghou,45
1,Beijing,Wuhan,54,Wuhan,45


- 我们可以看见l_df和r_df的相同名称的“key2”列分别被加上了“_x”和“_y”,我们可以自己修改

In [7]:
pd.merge(l_df,r_df,on='key1',suffixes =('_左边','_右边'))

,key1,Key2_左边,Value_左边,Key2_右边,Value_右边
0,Shanghai,Hanzghou,45,Hanzghou,45
1,Beijing,Wuhan,54,Wuhan,45


#### 索引上的合并

In [10]:
# 这个主要使用在表格的连接列出现在索引中的情况
r_df = pd.DataFrame({
    'Key2':["Hanzghou","Wuhan","Chengdu","Zhengzhou"],
    'Value':[45,45,78,90] 
},index = ['Shanghai','Beijing','Chenzhen','Guanzghou'])
r_df

,Key2,Value
Shanghai,Hanzghou,45
Beijing,Wuhan,45
Chenzhen,Chengdu,78
Guanzghou,Zhengzhou,90


In [11]:
pd.merge(l_df,r_df,left_on='key1',right_index = True)

,key1,Key2_x,Value_x,Key2_y,Value_y
0,Shanghai,Hanzghou,45,Hanzghou,45
1,Beijing,Wuhan,54,Wuhan,45


###  轴向连接
> 轴向连接更多的理解是组合表，这里就不需要字段一一对应关系，单纯的多合一
>- objs : 参与连接的pandas对象的列表或者字典，唯一的必须参数
>- axis : 指明连接的轴向，默认为0
>- join ：“inner”交集，“outer”并集，默认是并集
>- join_axes ：指明用于其他n-1条轴的索引，不执行交并集计算
>- keys : 用于形成层次化索引，简单书就是给连接的区域做个区分，下面举例
>- levels：如果设置了keys的话，指定用作层次化索引各级别上的索引
>- names: 用于创建分层级别的名称，如果设置了Keys或者levels的话
>- verify_integrity：检查结果对象新轴上的重复情况，如果发现则引发异常，默认False
>- ignore_index ：不保留连接轴上的索引，产生一组新的索引

In [15]:
import numpy as np
import pandas as pd
df1 = pd.DataFrame(np.arange(6).reshape(3,2),index = ['a','b','c'],
                  columns = ['Shanghai',"Beijing"])
df2 = pd.DataFrame(np.arange(7,11).reshape(2,2),index = ['a','c'],
                  columns = ["Guangdong","Chenzhen"])
pd.concat([df1,df2],axis=1,keys = ['level1','level2'],sort = False)

level1            level2         
  Shanghai Beijing Guangdong Chenzhen
a        0       1       7.0      8.0
b        2       3       NaN      NaN
c        4       5       9.0     10.0

> 这里可以看见df1和df2分别被用level1和level2区分开了

In [17]:
pd.concat([df1,df2],axis =1,keys = ["level1","level2"],
         names = ['upper',"lower"],
         sort =False)

upper   level1            level2         
lower Shanghai Beijing Guangdong Chenzhen
a            0       1       7.0      8.0
b            2       3       NaN      NaN
c            4       5       9.0     10.0

### 合并补丁数据
> 这个叫法也不知道是否同意，比如说我们有一张总表数据存在很多缺失，我们根据每天生成的小表根据索引去弥补这些缺失的数据，有数据的地方保持不变，填充缺值

In [23]:
a = pd.Series([np.nan,2.6,np.nan,3.6,4.6,np.nan],
             index =['f','e','d','c','b','a'])
b = pd.Series(np.arange(len(a)),dtype =np.float64,index = ['f','e','d','c','b','a'])
b[-1] =np.nan

# 方法一：使用where的方法
np.where(pd.isnull(a),b,a)

array([0. , 2.6, 2. , 3.6, 4.6, nan])

In [24]:
#  f方法2 ：combine_first方法，实现一样的功能，而且会进行数据的对齐
b[:-2].combine_first(a[2:])

a    NaN
b    4.6
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64